<a href="https://colab.research.google.com/github/AnnaZhuravleva/compling/blob/master/assignment_1/assignment_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Assignment 1

Using text http://www.gutenberg.org/files/2600/2600-0.txt
1. Make text lowercase and remove all punctuation except spaces and dots.
2. Tokenize text by BPE with vocab_size = 100
3. Train 3-gram language model with laplace smoothing $\delta=1$
4. Using beam search with k=10 generate sequences of length=10 conditioned on provided inputs. Treat dots as terminal tokens.
5. Calculate perplexity of the language model for the first sentence.

In [0]:
import string
import re
import collections
from collections import Counter
import nltk
from sklearn.base import TransformerMixin
import numpy as np


In [136]:

from google.colab import drive
drive.mount('/content/drive')
text = open('/content/drive/My Drive/Colab Notebooks/compling/assignment_1/peace.txt')
# text = open('peace.txt', 'r', encoding='utf-8-sig')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [137]:

text = text.read()
len(text)

3227581

In [138]:
def preprocess_text(text):
    punct = list(set(re.findall('\W', text)))
    punct.remove('.')
    words = ''
    for word in text.split():
        for letter in word:
            if letter not in punct:
                words += letter
        words += ' '
    text = words.lower()
        
    # replace all punctuation except dots with spaces
    # collapse multiple spaces into one '   ' -> ' '
    return text


text = preprocess_text(text)
len(text)
# assert len(text) == 3141169

3130570

In [0]:
text = text.split('.')
text = [x.strip() for x in text]

In [0]:
class BPE(TransformerMixin):
    
    def __init__(self, vocab_size=100):
        super(BPE, self).__init__()
        self.vocab_size = vocab_size
        # index to token
        self.itos = []
        # token to index
        self.stoi = {}
        
    def fit(self, text):
        """
        fit itos and stoi
        text: list of strings 
        """
        
        # TODO
        # tokenize text by symbols and fill in self.itos and self.stoi
        self.itos = list(set([item for txt in text for item in set(txt)]))
        self.stoi = {l: idx for idx, l in enumerate(self.itos)}
        text = [[self.stoi[letter] for letter in txt] for txt in text]
        
        while len(self.itos) < self.vocab_size:
            # TODO
            # count bigram freqencies in the text
            bigrams = collections.Counter()
            for txt in text:
                i = 0
                while i + 1 < len(txt):
                    bigrams[(txt[i], txt[i+1])] += 1
                    i += 1
                
                
            mc = bigrams.most_common(1)[0][0]
            new_token = str(self.itos[int(mc[0])]) + str(self.itos[int(mc[1])])
            new_id = len(self.itos)
            
            self.itos.append(new_token)
            self.stoi[new_token] = new_id
            
            # find occurences of the new_token in the text and replace them with new_id
            tmp = []
            for txt in text:
                tmp2 = []
                i = 0
                while i + 1 < len(txt):
                    if new_token == self.itos[txt[i]] + self.itos[txt[i+1]]:
                        tmp2.append(new_id)
                        i += 2
                    else:
                        tmp2.append(txt[i])
                        i += 1
                tmp.append(tmp2)
                    
            text = tmp 
           
        return self
    
    def transform(self, text):
        """
        convert text to a sequence of token ids
        text: list of strings
        """ 
        max_size = max([len(tok) for tok in self.itos])
        
        new_text = []
        for txt in text:
            i = 0
            new_txt = []
            while i < len(txt):
                hit = False
                stop = i+max_size if len(txt) - (i + max_size) >= 0 else len(txt)
                while hit == False and stop > i:
                    if txt[i:stop] in self.itos:
                        new_txt.append(self.stoi[txt[i:stop]])
                        hit = True
                        i = stop
                    else:
                        stop -= 1
            new_text.append(new_txt)
                                
        text = new_text
       # for token_id, token in enumerate(self.itos):
            # find occurences of the token in the text and replace them with token_id
          #  text = # TODO       
        return text
    
    def decode_token(self, tok):
        """
        tok: int or tuple
        """
        result = self.itos[tok] if isinstance(tok, int) else [self.itos[i] for i in token]
        return result
            
    def decode(self, text):
        """
        convert token ids into text
        """
        return ''.join(map(self.decode_token, text))
        
        
vocab_size = 100
bpe = BPE(vocab_size)
tokenized_text = bpe.fit_transform(text)

In [0]:
assert bpe.decode(tokenized_text[0]) == text[0]

In [143]:
bpe.decode(tokenized_text[0]) 

'the project gutenberg ebook of war and peace by leo tolstoy this ebook is for the use of anyone anywhere at no cost and with almost no restrictions whatsoever'

In [144]:
text[0]

'the project gutenberg ebook of war and peace by leo tolstoy this ebook is for the use of anyone anywhere at no cost and with almost no restrictions whatsoever'

In [145]:
print(tokenized_text[0])

[65, 38, 47, 15, 28, 13, 17, 61, 9, 55, 43, 69, 39, 64, 73, 13, 39, 15, 15, 32, 3, 83, 46, 82, 3, 71, 38, 13, 54, 17, 57, 39, 66, 19, 13, 68, 43, 15, 19, 89, 15, 66, 58, 5, 60, 13, 39, 15, 15, 32, 3, 5, 60, 25, 75, 3, 65, 55, 10, 57, 83, 63, 51, 67, 57, 63, 51, 46, 34, 64, 57, 84, 45, 68, 17, 15, 89, 3, 71, 99, 58, 3, 87, 23, 15, 89, 3, 45, 68, 79, 89, 47, 5, 17, 43, 5, 67, 60, 46, 78, 43, 10, 15, 13, 2, 64]


In [0]:
start_token = vocab_size
end_token = vocab_size + 1
        
    
class LM:
    
    def __init__(self, vocab_size, delta=1):
        self.delta = delta
        self.vocab_size = vocab_size + 2
        self.proba = Counter() # TODO create array for storing 3-gram counters
        
    def infer(self, a, b, tau=1):
        """
        return vector of probabilities of size self.vocab for 3-grams which start with (a,b) tokens
        a: first token id
        b: second token id
        tau: temperature
        """
        result = []
        for item in range(self.vocab_size):
            result.append(self.get_proba(a, b, item, tau))
        return result
        
    def get_proba(self, a, b, c, tau=1):
        """
        get probability of 3-gram (a,b,c)
        a: first token id
        b: second token id
        c: third token id
        tau: temperature
        """
        # TODO approbximate probability by counters
        
        P_abc = self.proba[a, b, c] + self.delta
        ab = [item for item in self.proba.keys() if item[0] == a \
              and item[1] == b]
        P_ab = sum([self.proba[item] for item in ab]) + self.delta
        n = 1/tau        
        result = (P_abc ** n) / (P_ab ** n) if P_abc > self.delta else 0 
        return result
    
    def fit(self, text):
        """
        train language model on text
        text: list of lists
        """
        self.text = text
        for txt in text:
            for i in range(len(txt) - 2):
                self.proba[txt[i], txt[i+1], txt[i+2]] += 1
                                       
        # self.proba = # TODO count 3-grams in the text
    
        return self
    
lm = LM(vocab_size, 1).fit(tokenized_text)

In [0]:
def beam_search(input_seq, lm, max_len=10, k=5, tau=1):
    """
    generate sequence from language model *lm* conditioned on input_seq
    input_seq: sequence of token ids for conditioning
    lm: language model
    max_len: max generated sequence length
    k: size of beam
    tau: temperature
    """
    # TODO store in beam tuples of current sequences and their log probabilities
    
   
    if len(input_seq) >= 3:
      prob = lm.get_proba(input_seq[0], input_seq[1] ,input_seq[2], tau)
      if len(input_seq) > 3:
        i = 1
        while i + 2 < len(input_seq):
          prob *= lm.get_proba(input_seq[i], input_seq[i + 1] ,input_seq[i + 2], tau)
          i += 1
      proba = enumerate(lm.infer(input_seq[-2], input_seq[-1], tau))
      proba = sorted(proba, key=lambda x: x[1], reverse=True)[:k]
      beam = {}
      for i in range(k):
        keys = list(input_seq)
        keys.append(proba[i][0])
        keys = tuple(list(keys))
        beam[keys] = proba[i][1] * prob
        
    elif len(input_seq) == 2:
      beam = [item for item in lm.proba.keys() if item[0] == input_seq[-2] \
                and item[1] == input_seq[-1]]
      beam = {item: lm.get_proba(item[0], item[1], item[2], tau) for item in beam}   
    else:
        beam = [item for item in lm.proba.keys() if item[0] == input_seq[0]]
        beam = {item: lm.get_proba(item[0], item[1], item[2], tau) for item in beam}   
            
    beam = sorted(beam.items(), key= lambda x: x[1], reverse=True)[:k]
    beam = {item[0] : item[1] for item in beam}
    
    for i in range(max_len):
      if sum ([1 for item in beam.keys() if item[-1] != '.']) == 0:
        break
      candidates = []
      candidates_proba = []
      for snt, snt_proba in beam.items():          
        key = list(snt)
        if sum(lm.infer(snt[-2], snt[-1], tau)) == 0 or lm.get_proba(snt[-3], snt[-2], snt[-1], tau) == 0:
          if snt[-1] != '.':
            key.append('.')
          candidates.append(tuple(key))
          candidates_proba.append(snt_proba)
            # TODO process terminal token нет нужной триграммы
        else:   
          proba = {i:j for i, j in enumerate(lm.infer(snt[-2], snt[-1]))} # probability vector of the next token
          best_k = sorted(proba.items(), key=lambda x: x[1], reverse=True)
          for item in best_k:
            candidates_proba.append(item[1] * snt_proba)
            key = list(snt)
            key.append(item[0])
            candidates.append(tuple(key))
      beam = {candidates[a]: candidates_proba[a] for a in range(len(candidates))}
      beam = sorted(beam.items(), key=lambda x: x[1], reverse=True)[:k]
      beam = {i[0]: i[1] for i in beam}
    #  print(i, beam)
                   
    return beam
   

In [148]:
input1 = 'horse '
input1 = bpe.transform([input1])[0]
result = beam_search(input1, lm, max_len=10, k=10, tau=0.1)
# TODO print decoded generated strings and their probabilities

for res in result.items():
  print(bpe.decode(tuple([i for i in res[0] if i != '.'])), res[1])
    

horse who had not but  3.417764459205561e-21
horse would not began 2.7155020053582248e-21
horse who had began to  2.685912601734959e-21
horse who had began w 2.5435268734502138e-21
horse who had began a 2.4852781664246364e-21
horse who had began e 2.213450866971942e-21
horse who had began o 1.9286794104024524e-21
horse who had not been  1.893387384939158e-21
horse who had bection  1.835230340237843e-21
horse who had began s 1.8121819963512976e-21


In [149]:
input1 = 'her'
input1 = bpe.transform([input1])[0]
result = beam_search(input1, lm, max_len=10, k=10, tau=0.1)
# TODO print decoded generated strings and their probabilities

for res in result.items():
  print(bpe.decode(tuple([i for i in res[0] if i != '.'])), res[1])

here who had began  2.6774229294880305e-08
here said not but  1.028099791898742e-08
here who had befor  9.435789990162033e-09
here who had not be 5.954611543549612e-09
here said not been  5.695510031993617e-09
here who had not be  4.2322032458286495e-09
here said not beg 3.923401611712994e-09
here who had not se 3.45478707330829e-09
here said not bef 3.304324434322383e-09
here said not bec 3.2772398078115443e-09


In [150]:
input1 = 'what'
input1 = bpe.transform([input1])[0]
result = beam_search(input1, lm, max_len=10, k=10, tau=1)
# TODO print decoded generated strings and their probabilities

for res in result.items():
  print(bpe.decode(tuple([i for i in res[0] if i != '.'])), res[1])

whatess marright of  4.8764265715746865e-08
whatess marrights  4.414791522798949e-08
whatess marrighter  4.339964547836255e-08
whatess marrighten 4.1154836229481734e-08
whatess marright and  3.739971226504315e-08
whatess marrignich 3.59769268468042e-08
whatess marright to  3.5953314553135396e-08
whatess marright b 3.574668630857715e-08
whatedid not began 3.569629189896679e-08
whatess marrightl 3.1427329484331503e-08


In [151]:
input1 = 'gun '
input1 = bpe.transform([input1])[0]
result = beam_search(input1, lm, max_len=10, k=10, tau=0.1)
# TODO print decoded generated strings and their probabilities

for res in result.items():
  print(bpe.decode(tuple([i for i in res[0] if i != '.'])), res[1])

gun and said not know 3.2027464526453345e-36
gun and said not up  1.660597194750835e-36
gun and said not but  1.4810064882361813e-36
gun and said not kne 1.4781906704516927e-36
gun and said not been  8.20454140508919e-37
gun and said not und 7.883887164228773e-37
gun and said not loo 6.735291230741083e-37
gun and said not beg 5.651769690733994e-37
gun and said not bef 4.7599717119199515e-37
gun and said not bec 4.720955550346837e-37


In [0]:
def perplexity(snt, lm):
    """
    snt: sequence of token ids
    lm: language model
    """
    result = #TODO perplexity for the sentence
    return result

perplexity(tokenized_text[0], lm)